# Test model
Read stored model and test on custom dataset\
Sep 1, 2020

In [1]:
import numpy as np
#import pandas as pd
import time
import argparse
import sys
import subprocess as sp
import pickle
import yaml

## M-L modules
import tensorflow.keras
from tensorflow.keras import layers, models, optimizers, callbacks  # or tensorflow.keras as keras
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve, auc, roc_auc_score
from tensorflow.keras.models import load_model

## modules from other files
from models import *
from utils import dataset, cnn_model, f_get_data, f_load_config


In [2]:
model_save_dir='/global/cfs/cdirs/dasrepo/vpa/supernova_cnn/data/results_data/results/test_with_new_labels/'
data_dir='/global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/input_npy_files/'
prefix='full'


In [3]:
## Create dataset using indices

#### Read data from files 
data_dict=f_get_data(prefix,data_dir,pre_norm=False)

t1=time.time()

size_data=data_dict['labels'].shape[0]
print("Size of entire dataset is : ",size_data)
#### Define the indices for training, validation and test data
train_size,val_size,test_size=int(0.5*size_data),int(0.05*size_data),int(0.05*size_data)
# train_size,val_size,test_size=int(0.0*size_data),int(0.0*size_data),int(1.0*size_data)


### Get random indices for test,train,val
np.random.seed(225) # Set random seed
test_idx=np.random.choice(np.arange(size_data),test_size,replace=False)
# get remaining indices without test indices
rem_idx1=np.array(list(set(np.arange(size_data))-set(test_idx)))
val_idx=np.random.choice(rem_idx1,val_size,replace=False)
rem_idx2=np.array(list(set(rem_idx1)-set(val_idx)))
train_idx=np.random.choice(rem_idx2,train_size,replace=False)

print("Shapes of indices",train_idx.shape, val_idx.shape, test_idx.shape)

#### Storing arrays into train,validation, test objects and deleting the full data dictionary
train_data=dataset('training',data_dict,train_idx)
val_data=dataset('validation',data_dict,val_idx)
test_data=dataset('test',data_dict,test_idx)
del data_dict
# print("\nData shapes: Train {0}, Validation {1}, Test {2}\n".format(train_data.x.shape,val_data.x.shape,test_data.x.shape))

t2=time.time()
print("Time taken to read and process input files",t2-t1)

Input file name /global/project/projectdirs/dasrepo/vpa/supernova_cnn/data/gathered_data/input_npy_files/full_x.npy
Size of entire dataset is :  898963
Shapes of indices (449481,) (44948,) (44948,)
Time taken to read and process input files 18.613744258880615


In [4]:
### Checking that order of stored ids is same as that in the dictionary
test_ids=np.loadtxt(model_save_dir+'id_test_3.test')
np.array_equal(test_ids,test_data.id)

True

In [5]:
## Relabel target (y)
new_test_y=np.loadtxt(model_save_dir+'ytest_3.test') 
test_data.y=new_test_y ### Relabel


In [6]:
data=test_data

In [7]:
# model_name='3'
for model_name in [str(i) for i in [3,8,9,16]]:
    print(model_name)
    ### Define Object for cnn_model
    Model=cnn_model(model_name,model_save_dir)

    ### Read stored model and history
    Model.f_load_model_history()

    #################################
    ### Test model ###
    Model.f_test_model(data)

    ## Save prediction array and labels array
    Model.f_save_predictions(data,val_data,val_data)

3


W1103 06:49:01.206565 46912496688512 deprecation.py:506] From /global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1103 06:49:01.223641 46912496688512 deprecation.py:506] From /global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1103 06:49:01.241270 46912496688512 deprecation.py:506] From /global/homes/v/vpa/.conda/envs/v_py3/lib/python3.6/site-packages/tensorflow_core/python/ops/init_

(44948, 51, 51, 3)
44948/44948 [==============================] - 5s 108us/sample
8
(44948, 51, 51, 3)
44948/44948 [==============================] - 5s 111us/sample
9
(44948, 51, 51, 3)
44948/44948 [==============================] - 7s 152us/sample
16
(44948, 51, 51, 3)
44948/44948 [==============================] - 3s 62us/sample
